Synchronous programming example

In [13]:
import time

def do_something(obj):
    print(f"Invoking.. {obj}")
    time.sleep(obj)
    print(f"Done with {obj}")
    return f"Result of {obj}"

def main():
    task1 = do_something(1)
    print(f"Done with task1")
    task2 = do_something(2)
    print(f"Done with task2")
    return [task1, task2]

t1 = time.perf_counter()
results = main()
t2 = time.perf_counter()
print(results)

print(f"Completed in {t2 - t1:.2f} sec")

Invoking.. 1
Done with 1
Done with task1
Invoking.. 2
Done with 2
Done with task2
['Result of 1', 'Result of 2']
Completed in 3.02 sec


Asynchronous example - using tasks

 - here we create two tasks that invoke and async function which has some To-do


In [ ]:
import time
import asyncio

async def do_something(obj):
    print(f"Invoking.. {obj}")
    await asyncio.sleep(obj)
    print(f"Done with {obj}")
    return f"Result of {obj}"

async def main():
    task1 = asyncio.create_task(do_something(3))
    task2 = asyncio.create_task(do_something(2))
    result1 = await task1
    print(f"Done with task1")
    result2 = await task2
    print(f"Done with task2")
    return [result1, result2]

t1 = time.perf_counter()

results = await main()      # we can also use -> asyncio.run(main())
print(results)

t2 = time.perf_counter()


print(f"Completed in {t2 - t1:.2f} sec")

Invoking.. 3
Invoking.. 2
Done with 2
Done with 3
Done with task1
Done with task2
['Result of 3', 'Result of 2']
Completed in 3.02 sec


Below example is continuation of above, however, in this we use string comprehension to create the tasks and use asyncio.gather() group the await for all tasks.
In this I have tried to produce an error purposefully to understand what happens if some issue occurs with one of the task

so I am passing a list i.e. [1,'a',2] to the asyncio.create_task method, this will create 3 tasks one for each list item.
The issue occurs when string element of list is passed to asyncio.sleep() method which expects only an int.

Here we can see, after our error, the main method gets stopped at the await step.


In [5]:
import time
import asyncio

async def do_something(obj):
    print(f"Invoking.. {obj}")
    await asyncio.sleep(obj)
    print(f"Done with {obj}")
    return f"Result of {obj}"

async def main():
    a = [1,'a',2]
    task = [asyncio.create_task(do_something(i)) for i in a] 
    result = await asyncio.gather(*task)
    print(f"Done with task1")
    return result

t1 = time.perf_counter()

results = await main()              # we can also use -> asyncio.run(main()) 
print(results)

t2 = time.perf_counter()


print(f"Completed in {t2 - t1:.2f} sec")

Invoking.. 1
Invoking.. a
Invoking.. 2


TypeError: '<=' not supported between instances of 'str' and 'int'

Done with 1
Done with 2


Now we can handle the above error by 
 - either handling exception in our task function (i.e. do_something)
 - Or by passing an argument return_exception=True in asyncio.gather() method

In [6]:
import time
import asyncio

async def do_something(obj):
    print(f"Invoking.. {obj}")
    await asyncio.sleep(obj)
    print(f"Done with {obj}")
    return f"Result of {obj}"

async def main():
    a = [1,'a',2]
    task = [asyncio.create_task(do_something(i)) for i in a] 
    result = await asyncio.gather(*task, return_exceptions=True)
    print(f"Done with task1")
    return result

t1 = time.perf_counter()

results = await main()
print(results)

t2 = time.perf_counter()


print(f"Completed in {t2 - t1:.2f} sec")

Invoking.. 1
Invoking.. a
Invoking.. 2
Done with 1
Done with 2
Done with task1
['Result of 1', TypeError("'<=' not supported between instances of 'str' and 'int'"), 'Result of 2']
Completed in 2.02 sec


Handling a syncronous task using asynchronous methods :

Here instead of create_task, we use to_thread method, this will create asynchronous threads which will handle the syncronous tasks.

In [9]:
import time
import asyncio

def do_something(obj):
    print(f"Invoking.. {obj}", flush=True)
    time.sleep(obj)
    print(f"Done with {obj}", flush=True)
    return f"Result of {obj}"

async def main():
    a = [1,3,2]
    task = [asyncio.to_thread(do_something, i) for i in a] 
    result = await asyncio.gather(*task)
    print(f"Done with task")
    return result

t1 = time.perf_counter()

results = await main()              # we can also use -> asyncio.run(main()) 
print(results)

t2 = time.perf_counter()


print(f"Completed in {t2 - t1:.2f} sec")

Invoking.. 1Invoking.. 3
Invoking.. 2

Done with 1
Done with 2
Done with 3
Done with task
['Result of 1', 'Result of 3', 'Result of 2']
Completed in 3.02 sec
